# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/$PROJECT_ID/fltk`
    * Push to your gcr.io repository `docker push gcr.io/$PROJECT_ID/fltk`


With that setup, first set some variables used throughout the experiment.


In [1]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="test-bed-fltk-group16-mb"
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL="medium-fltk-pool-1"
REGION="us-central1-c"

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update



Scale the default-node-pool up.

In [2]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].


## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [3]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test flearner

W1027 16:02:16.273327    7975 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: flearner: release: not found


: 1

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [4]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1027 16:02:23.269780    7997 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" does not exist. Installing it now.
NAME: extractor
LAST DEPLOYED: Thu Oct 27 16:02:24 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [25]:
# HOMEWORKS
# EXPERIMENT_FILE="/home/mattiacs/Documents/Developer/Edu/TUDelft/Year_4/Quarter_1/CS4215-QPECS/homeworks_2/ex6_configs/distributed_4_node.json"

# PROJECT
# EXPERIMENT_FILE="../configs/federated_tasks/example_arrival_config.json"
# EXPERIMENT_FILE="../configs/distributed_tasks/example_arrival_config.json"
# CLUSTER_CONFIG="../configs/example_cloud_experiment.json"

EXPERIMENT_FILE="../project/configs/queue/experiment_m_m_1_fast.json"
CLUSTER_CONFIG="../project/configs/project_cloud_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [26]:
helm uninstall -n test experiment-orchestrator
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1027 16:29:39.224135    9872 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled
W1027 16:29:43.234933    9893 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Thu Oct 27 16:29:44 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [27]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

W1027 16:30:07.482599    9925 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-27-2022 14:29:52 root         INFO     Loading file config/configuration.fltk.json
No argument path is provided.
10-27-2022 14:29:52 root         INFO     Starting in cluster mode.
10-27-2022 14:29:52 root         INFO     Starting with experiment replication: 0 with seed: 42
10-27-2022 14:29:52 root         INFO     Starting as Orchestrator
10-27-2022 14:29:52 root         INFO     Starting Orchestrator, initializing resources....
10-27-2022 14:29:52 root         INFO     Loading in cluster configuration file
10-27-2022 14:29:52 root         INFO     Pointing configuration to in cluster configuration.
10-27-2022 14:29:52 root         INFO     Starting cluster manager
10-27-2022 14:29:52 ClusterManager INFO     Spinning up cluster manag

In [46]:
CURRENT_POD_ID="16f9d108-c0e0-4bf3-90c2-387135088ec2"
# To get logs from learners (example)
kubectl logs -n test trainjob-$CURRENT_POD_ID-master-0

W1027 17:05:44.727806   12370 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-27-2022 14:30:14 root         INFO     Loading file config/configuration.fltk.json
10-27-2022 14:30:14 root         INFO     Starting in client mode
10-27-2022 14:30:14 root         INFO     Starting with host=trainjob-16f9d108-c0e0-4bf3-90c2-387135088ec2-master-0 and port=23456
10-27-2022 14:30:14 root         INFO     Starting Creating client with 0
10-27-2022 14:30:14 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     Initializing learning client
10-27-2022 14:30:14 root         INFO     Getting net: Nets.fashion_mnist_cnn
10-27-2022 14:30:14 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     Preparing learner model with distributed=False
10-27-2022 14:30:14 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [1,     0] lo

10-27-2022 14:40:32 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [8,   250] loss: 0.211
10-27-2022 14:40:40 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [8,   300] loss: 0.216
10-27-2022 14:40:48 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [8,   350] loss: 0.223
10-27-2022 14:40:56 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [8,   400] loss: 0.215
10-27-2022 14:41:04 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [8,   450] loss: 0.222
10-27-2022 14:41:13 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [9,     0] loss: 0.003
10-27-2022 14:41:21 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [9,    50] loss: 0.190
10-27-2022 14:41:29 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [9,   100] loss: 0.211
10-27-2022 14:41:38 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [9,   150] loss: 0.213
10-27-2022 14:41:47 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [9,   200] loss: 0.200
10-27-2022 14:41:55 

10-27-2022 14:52:41 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [16,   400] loss: 0.171
10-27-2022 14:52:49 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [16,   450] loss: 0.159
10-27-2022 14:53:00 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [17,     0] loss: 0.003
10-27-2022 14:53:09 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [17,    50] loss: 0.153
10-27-2022 14:53:18 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [17,   100] loss: 0.148
10-27-2022 14:53:26 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [17,   150] loss: 0.150
10-27-2022 14:53:34 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [17,   200] loss: 0.146
10-27-2022 14:53:42 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [17,   250] loss: 0.162
10-27-2022 14:53:51 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [17,   300] loss: 0.160
10-27-2022 14:53:59 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [17,   350] loss: 0.154
10-27-2022

10-27-2022 15:04:32 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [25,    50] loss: 0.107
10-27-2022 15:04:40 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [25,   100] loss: 0.109
10-27-2022 15:04:47 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [25,   150] loss: 0.134
10-27-2022 15:04:55 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [25,   200] loss: 0.113
10-27-2022 15:05:03 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [25,   250] loss: 0.113
10-27-2022 15:05:11 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [25,   300] loss: 0.129
10-27-2022 15:05:18 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [25,   350] loss: 0.121
10-27-2022 15:05:26 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [25,   400] loss: 0.119
10-27-2022 15:05:34 Client-0-16f9d108-c0e0-4bf3-90c2-387135088ec2 INFO     [25,   450] loss: 0.123
No argument path is provided.
[EpochData(epoch_id=1, num_epochs=26, duration_train=75852, duration_test=7031,

       0.99378882, 0.73904179, 0.94449854, 0.98590131, 0.95948617]), class_recall=array([0.853, 0.979, 0.839, 0.943, 0.819, 0.96 , 0.725, 0.97 , 0.979,
       0.971]), confusion_mat=array([[853,   0,  19,  29,   2,   1,  94,   0,   2,   0],
       [  1, 979,   0,  15,   2,   0,   2,   0,   1,   0],
       [ 20,   1, 839,  14,  61,   0,  64,   0,   1,   0],
       [ 12,   3,   8, 943,  10,   0,  24,   0,   0,   0],
       [  1,   1,  57,  50, 819,   0,  71,   0,   1,   0],
       [  0,   0,   0,   0,   0, 960,   0,  27,   1,  12],
       [125,   0,  60,  32,  50,   0, 725,   0,   8,   0],
       [  0,   0,   0,   0,   0,   1,   0, 970,   0,  29],
       [  3,   0,   2,   5,   4,   1,   1,   5, 979,   0],
       [  1,   0,   0,   0,   0,   3,   0,  25,   0, 971]])), EpochData(epoch_id=9, num_epochs=26, duration_train=736964, duration_test=7094, loss_train=0.20382326260209083, accuracy=89.82, loss=21.47817126661539, class_precision=array([0.8875    , 0.98988878, 0.86847599, 0.90394338, 0.

       0.974]), confusion_mat=array([[864,   0,  12,  18,   7,   1,  94,   0,   4,   0],
       [  3, 983,   1,   9,   2,   0,   2,   0,   0,   0],
       [ 24,   1, 779,   7, 123,   0,  66,   0,   0,   0],
       [ 24,   5,   7, 882,  58,   0,  24,   0,   0,   0],
       [  2,   1,  21,  12, 929,   0,  35,   0,   0,   0],
       [  0,   0,   0,   0,   0, 980,   0,  14,   0,   6],
       [118,   0,  39,  25,  86,   0, 726,   0,   6,   0],
       [  0,   0,   0,   0,   0,   8,   0, 971,   0,  21],
       [  5,   1,   1,   1,   5,   1,   4,   4, 977,   1],
       [  1,   0,   0,   0,   0,   3,   0,  22,   0, 974]])), EpochData(epoch_id=17, num_epochs=26, duration_train=1444767, duration_test=7402, loss_train=0.14771497800946234, accuracy=90.49, loss=22.046285338699818, class_precision=array([0.89354473, 0.99691992, 0.80849142, 0.86464088, 0.90625   ,
       0.98293173, 0.7150838 , 0.96307385, 0.97914598, 0.96722939]), class_recall=array([0.789, 0.971, 0.895, 0.939, 0.783, 0.979, 0.768, 0

       [ 10,   2, 843,   3,  86,   0,  55,   0,   1,   0],
       [ 29,  14,  19, 826,  59,   0,  51,   0,   2,   0],
       [  1,   0,  41,   8, 911,   0,  37,   0,   2,   0],
       [  0,   0,   0,   0,   0, 980,   0,  13,   0,   7],
       [ 80,   1,  53,  15,  88,   0, 754,   0,   9,   0],
       [  0,   0,   0,   0,   0,  10,   0, 966,   0,  24],
       [  1,   1,   1,   1,   3,   2,   2,   2, 987,   0],
       [  0,   0,   0,   0,   0,   3,   0,  23,   1, 973]])), EpochData(epoch_id=25, num_epochs=26, duration_train=2122568, duration_test=6804, loss_train=0.12295131735503674, accuracy=91.22, loss=22.49334315955639, class_precision=array([0.82544104, 0.99391481, 0.88461538, 0.8861244 , 0.84955752,
       0.97813121, 0.77814208, 0.96777442, 0.9879397 , 0.96812749]), class_recall=array([0.889, 0.98 , 0.851, 0.926, 0.864, 0.984, 0.712, 0.961, 0.983,
       0.972]), confusion_mat=array([[889,   0,  12,  18,   4,   1,  72,   0,   4,   0],
       [  4, 980,   0,  13,   1,   0,   2,   0,

In [23]:
# To get logs from learners (federated learning)
kubectl logs -n test trainjob-$CURRENT_POD_ID-worker-0

W1027 16:28:43.916849    9780 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-27-2022 14:13:25 root         INFO     Loading file config/configuration.fltk.json
10-27-2022 14:13:25 root         INFO     Starting in client mode
10-27-2022 14:13:25 root         INFO     Starting with host=trainjob-755cc21b-8714-4238-9bb2-ca420cb34c08-master-0 and port=23456
10-27-2022 14:13:25 root         INFO     Initializing backend for training process: gloo
10-27-2022 14:13:25 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 1
10-27-2022 14:13:25 torch.distributed.distributed_c10d INFO     Rank 1: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-27-2022 14:13:25 root         INFO

10-27-2022 14:24:43 Client-1-755cc21b-8714-4238-9bb2-ca420cb34c08 INFO     [24,     0] loss: 0.003
10-27-2022 14:24:54 Client-1-755cc21b-8714-4238-9bb2-ca420cb34c08 INFO     [24,    50] loss: 0.113
10-27-2022 14:25:07 Client-1-755cc21b-8714-4238-9bb2-ca420cb34c08 INFO     [24,   100] loss: 0.100
10-27-2022 14:25:13 Client-1-755cc21b-8714-4238-9bb2-ca420cb34c08 INFO     [25,     0] loss: 0.003
10-27-2022 14:25:26 Client-1-755cc21b-8714-4238-9bb2-ca420cb34c08 INFO     [25,    50] loss: 0.103
10-27-2022 14:25:38 Client-1-755cc21b-8714-4238-9bb2-ca420cb34c08 INFO     [25,   100] loss: 0.097
No argument path is provided.
[EpochData(epoch_id=1, num_epochs=26, duration_train=27228, duration_test=2166, loss_train=0.6092325168848037, accuracy=79.68, loss=10.855425238609314, class_precision=array([0.775     , 0.98062016, 0.58196721, 0.78745645, 0.72988506,
       0.73888889, 0.65806452, 0.93055556, 0.93721973, 0.94570136]), class_recall=array([0.78151261, 0.94052045, 0.89121339, 0.87937743, 0.55

       0.96678967, 0.61904762, 0.92395437, 0.97008547, 0.97142857]), class_recall=array([0.73529412, 0.96282528, 0.76150628, 0.90661479, 0.78695652,
       0.97761194, 0.81889764, 0.95294118, 0.96186441, 0.93700787]), confusion_mat=array([[175,   1,   2,   7,   1,   0,  52,   0,   0,   0],
       [  0, 259,   1,   5,   1,   0,   3,   0,   0,   0],
       [  1,   0, 182,   1,  24,   0,  31,   0,   0,   0],
       [  0,   3,   4, 233,   1,   0,  15,   0,   1,   0],
       [  0,   0,  13,   9, 181,   0,  27,   0,   0,   0],
       [  0,   0,   0,   1,   0, 262,   0,   4,   0,   1],
       [ 13,   1,   8,   9,  10,   0, 208,   0,   5,   0],
       [  0,   0,   0,   0,   0,   6,   0, 243,   0,   6],
       [  0,   1,   2,   1,   1,   3,   0,   1, 227,   0],
       [  0,   0,   0,   0,   0,   0,   0,  15,   1, 238]])), EpochData(epoch_id=9, num_epochs=26, duration_train=264635, duration_test=2155, loss_train=0.2583915534615517, accuracy=88.28, loss=6.337469518184662, class_precision=array([0

       0.97358491, 0.62345679, 0.91698113, 0.97446809, 0.96370968]), class_recall=array([0.78151261, 0.95910781, 0.80753138, 0.91439689, 0.71304348,
       0.96268657, 0.79527559, 0.95294118, 0.97033898, 0.94094488]), confusion_mat=array([[186,   1,   4,   4,   0,   0,  43,   0,   0,   0],
       [  1, 258,   2,   6,   1,   0,   1,   0,   0,   0],
       [  1,   0, 193,   2,  14,   0,  28,   0,   1,   0],
       [  2,   3,   5, 235,   0,   0,  12,   0,   0,   0],
       [  0,   0,  18,  10, 164,   0,  38,   0,   0,   0],
       [  0,   0,   0,   1,   0, 258,   0,   7,   0,   2],
       [ 21,   1,   8,  11,   7,   0, 202,   0,   4,   0],
       [  0,   0,   0,   0,   0,   5,   0, 243,   0,   7],
       [  0,   0,   1,   2,   1,   2,   0,   1, 229,   0],
       [  0,   0,   0,   0,   0,   0,   0,  14,   1, 239]])), EpochData(epoch_id=17, num_epochs=26, duration_train=497606, duration_test=2174, loss_train=0.1622651143372059, accuracy=88.44, loss=6.452883675694466, class_precision=array([

       0.97368421, 0.71875   , 0.91078067, 0.97446809, 0.97933884]), class_recall=array([0.85714286, 0.96654275, 0.78661088, 0.92217899, 0.82608696,
       0.96641791, 0.72440945, 0.96078431, 0.97033898, 0.93307087]), confusion_mat=array([[204,   1,   5,   5,   0,   0,  23,   0,   0,   0],
       [  2, 260,   0,   6,   1,   0,   0,   0,   0,   0],
       [  4,   0, 188,   3,  25,   0,  18,   0,   1,   0],
       [  2,   3,   4, 237,   2,   0,   9,   0,   0,   0],
       [  0,   0,  10,   8, 190,   0,  22,   0,   0,   0],
       [  0,   0,   0,   1,   0, 259,   0,   6,   0,   2],
       [ 32,   0,   9,  15,  10,   0, 184,   0,   4,   0],
       [  0,   0,   0,   0,   0,   7,   0, 245,   0,   3],
       [  1,   0,   1,   2,   1,   0,   0,   2, 229,   0],
       [  0,   0,   0,   0,   0,   0,   0,  16,   1, 237]])), EpochData(epoch_id=25, num_epochs=26, duration_train=737123, duration_test=2161, loss_train=0.09712781667709351, accuracy=89.16, loss=6.895568206906319, class_precision=array(

In [24]:
kubectl logs -n test trainjob-$CURRENT_POD_ID-worker-1

W1027 16:28:49.280707    9795 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-27-2022 14:13:23 root         INFO     Loading file config/configuration.fltk.json
10-27-2022 14:13:23 root         INFO     Starting in client mode
10-27-2022 14:13:23 root         INFO     Starting with host=trainjob-755cc21b-8714-4238-9bb2-ca420cb34c08-master-0 and port=23456
10-27-2022 14:13:23 root         INFO     Initializing backend for training process: gloo
10-27-2022 14:13:25 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 2
10-27-2022 14:13:25 torch.distributed.distributed_c10d INFO     Rank 2: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-27-2022 14:13:25 root         INFO

10-27-2022 14:24:43 Client-2-755cc21b-8714-4238-9bb2-ca420cb34c08 INFO     [24,     0] loss: 0.003
10-27-2022 14:24:54 Client-2-755cc21b-8714-4238-9bb2-ca420cb34c08 INFO     [24,    50] loss: 0.113
10-27-2022 14:25:07 Client-2-755cc21b-8714-4238-9bb2-ca420cb34c08 INFO     [24,   100] loss: 0.100
10-27-2022 14:25:13 Client-2-755cc21b-8714-4238-9bb2-ca420cb34c08 INFO     [25,     0] loss: 0.003
10-27-2022 14:25:26 Client-2-755cc21b-8714-4238-9bb2-ca420cb34c08 INFO     [25,    50] loss: 0.103
10-27-2022 14:25:38 Client-2-755cc21b-8714-4238-9bb2-ca420cb34c08 INFO     [25,   100] loss: 0.097
No argument path is provided.
[EpochData(epoch_id=1, num_epochs=26, duration_train=27233, duration_test=2063, loss_train=0.6092325168848037, accuracy=78.6, loss=12.075407087802887, class_precision=array([0.78787879, 1.        , 0.52331606, 0.74662162, 0.75121951,
       0.71165644, 0.66896552, 0.94977169, 0.9375    , 0.96698113]), class_recall=array([0.74590164, 0.97391304, 0.86324786, 0.87698413, 0.590

       0.958159  , 0.60117302, 0.93846154, 0.9688716 , 0.97942387]), class_recall=array([0.69262295, 0.9826087 , 0.74358974, 0.90873016, 0.76628352,
       0.97033898, 0.76208178, 0.96062992, 0.91881919, 0.95582329]), confusion_mat=array([[169,   0,   3,  13,   2,   0,  55,   0,   2,   0],
       [  0, 226,   1,   2,   0,   0,   1,   0,   0,   0],
       [  0,   0, 174,   5,  27,   0,  28,   0,   0,   0],
       [  0,   0,   1, 229,  12,   0,  10,   0,   0,   0],
       [  0,   1,  15,  13, 200,   0,  32,   0,   0,   0],
       [  0,   0,   0,   0,   0, 229,   0,   5,   0,   2],
       [ 22,   0,  11,  12,  13,   0, 205,   0,   6,   0],
       [  0,   0,   0,   0,   0,   7,   0, 244,   0,   3],
       [  3,   0,   3,   3,   0,   2,   9,   2, 249,   0],
       [  0,   0,   0,   0,   0,   1,   1,   9,   0, 238]])), EpochData(epoch_id=9, num_epochs=26, duration_train=264640, duration_test=2163, loss_train=0.2583915534615517, accuracy=86.76, loss=7.244910195469856, class_precision=array([0

       0.97413793, 0.65217391, 0.93283582, 0.96946565, 0.98353909]), class_recall=array([0.75409836, 0.97826087, 0.81196581, 0.92460317, 0.73563218,
       0.95762712, 0.78066914, 0.98425197, 0.93726937, 0.95983936]), confusion_mat=array([[184,   0,   6,   8,   1,   0,  44,   0,   1,   0],
       [  0, 225,   1,   3,   0,   0,   0,   0,   1,   0],
       [  1,   0, 190,   5,  16,   0,  22,   0,   0,   0],
       [  1,   0,   3, 233,   8,   0,   7,   0,   0,   0],
       [  0,   1,  19,  17, 192,   0,  32,   0,   0,   0],
       [  0,   0,   0,   0,   0, 226,   0,   7,   0,   3],
       [ 18,   0,  11,  11,  13,   0, 210,   0,   6,   0],
       [  0,   0,   0,   0,   0,   3,   0, 250,   0,   1],
       [  1,   0,   2,   3,   0,   2,   7,   2, 254,   0],
       [  0,   0,   0,   0,   0,   1,   0,   9,   0, 239]])), EpochData(epoch_id=17, num_epochs=26, duration_train=497611, duration_test=2178, loss_train=0.1622651143372059, accuracy=88.12, loss=7.294232413172722, class_precision=array([

       0.97835498, 0.73461538, 0.91911765, 0.97718631, 0.9832636 ]), class_recall=array([0.80327869, 0.97826087, 0.74786325, 0.92857143, 0.81226054,
       0.95762712, 0.71003717, 0.98425197, 0.94833948, 0.9437751 ]), confusion_mat=array([[196,   0,   7,  11,   3,   0,  26,   0,   1,   0],
       [  0, 225,   1,   3,   0,   0,   0,   0,   1,   0],
       [  5,   0, 175,   4,  35,   0,  15,   0,   0,   0],
       [  1,   0,   1, 234,  11,   0,   5,   0,   0,   0],
       [  0,   1,  15,  15, 212,   0,  18,   0,   0,   0],
       [  0,   0,   0,   0,   0, 226,   0,   7,   0,   3],
       [ 29,   0,  13,  15,  17,   0, 191,   0,   4,   0],
       [  0,   0,   0,   0,   0,   3,   0, 250,   0,   1],
       [  2,   0,   1,   2,   1,   1,   5,   2, 257,   0],
       [  0,   0,   0,   0,   0,   1,   0,  13,   0, 235]])), EpochData(epoch_id=25, num_epochs=26, duration_train=737128, duration_test=2062, loss_train=0.09712781667709351, accuracy=87.64, loss=8.144368901848793, class_precision=array(

## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [131]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")

kubectl cp -n test $EXTRACTOR_POD_NAME:/opt/federation-lab/logging ../logging

W1025 11:17:18.596194   23801 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
W1025 11:17:19.304109   23815 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
tar: Removing leading `/' from member names


# Cleanup

## Removing orchestrator

In [132]:
helm uninstall -n test experiment-orchestrator

W1025 11:17:29.586870   23842 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [133]:
helm uninstall extractor -n test

W1025 11:17:36.310075   23857 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [134]:
# This will remove all information and logs as well.
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

W1025 11:18:01.975271   23914 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
pytorchjob.kubeflow.org "trainjob-06a8547e-765a-4688-a76a-e58e330bdb43" deleted
pytorchjob.kubeflow.org "trainjob-0c0de4ab-dd8d-4589-b096-09802308e755" deleted
pytorchjob.kubeflow.org "trainjob-13ecb4e5-dc83-4e5b-a4cb-fe09b427f500" deleted
pytorchjob.kubeflow.org "trainjob-4505c753-eca9-4590-b94e-ae8be7ffe189" deleted
pytorchjob.kubeflow.org "trainjob-4d34cc70-effa-4fc6-bcdc-bb52ff6a9e6c" deleted
pytorchjob.kubeflow.org "trainjob-73b5efaa-d69b-473d-ad4b-36a99928a3ee" deleted
pytorchjob.kubeflow.org "trainjob-7560abd5-6ee8-41c2-b0c7-860c2b17f6e3" deleted
pytorchjob.kubeflow.org "trainjob-7fc7f5ca-0724-4e02-b7fb-cd3a5df1776b" deleted
pytorchjob.kubeflow.org "trainjob-81e860db-7686-45c8-bf56-fdf2c5f9daa2" deleted
pytorchjob.kubeflow.org "trai